# Digital Twin Data Acquisition

In this notebook, we will use Big Query and The Graph to obtain the signals required for the Rai Digital Twin. We will get state snapshots and calculate the absolute changes between time periods. 

# TODO: Define features below, add time frequency analysis - add readmes and document

### Signals  
* Price signals:
    * Eth price
    * Rai price in eth - need new source
    * Rai price in usd - need new source
    * Redemption price
    * Redemption rate.
* Uniswap data - https://thegraph.com/explorer/subgraph/uniswap/uniswap-v2
* Total debt = sum over SAFEs of debt
* Total collateral = sum over SAFEs of Collateral
* Total rai = directly observable from the RAI erc 20 contract (not sure how to get it from theGraph), further note that total rai = principle debt
* Excess Liability = Total debt - total rai
* ETH in Uniswap = should be queriable from thegraph data on uniswap (different source subgraph) --> future replace this with total value in ETH of assets RAI is paired with on Uniswap instances
* RAI in Uniswap = should be queriable from thegraph data on uniswap
* Debt ceiling = directly observable from RAI subgraph
* debt as fraction of debt ceiling = total debt/debt ceiling
* floating RAI = total rai - rai in uniswap
* floating RAI as fraction of total RAI = floating RAI /total RAI
another thing i am interested in from uniswap is the yield per unit liquidity
* total liquidity = price_usd_asset1 * quantity_of_asset1 + price_usd_asset2*quantity_of_asset2
* volume_per_period = take directly from contract
* fees_per_period = measure directly or compute from volumes
* revenues in token1 and token2
* value of profits in USD
* yields = profit in usd per total liquidity in USD of the whole pool
* Ratio of yields to total liquidity

In [1]:
# import libraries
import os
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import json
import requests
# The Graph URL
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'

%matplotlib inline

# constants
constant = 1000000000000000000

#defining creditionals
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/aclarkdata/Downloads/raidata-5e1723d42cf5.json'

# initializing agent
client = bigquery.Client()


In [2]:
# SQL query
sql = """
SELECT * 
FROM `blockchain-etl.ethereum_rai.ChainlinkMedianETHUSD_event_UpdateResult`
ORDER By block_timestamp DESC

"""

eth_price_next_OSM = client.query(sql).to_dataframe()
eth_price_next_OSM['ETH Price (Next OSM)'] = eth_price_next_OSM['medianPrice'].astype(float)/constant
# subset
eth_price_next_OSM = eth_price_next_OSM[['block_number','ETH Price (Next OSM)']]

eth_price_next_OSM.head()

,block_number,ETH Price (Next OSM)
0,12271233,2129.458042
1,12270946,2219.590000
2,12270590,2244.796301
3,12270259,2236.288682
4,12269966,2218.569789


In [3]:
# SQL query
sql = """
SELECT 
* 
FROM `blockchain-etl.ethereum_rai.OSM_event_UpdateResult`
ORDER By block_timestamp DESC

"""

eth_price_OSM = client.query(sql).to_dataframe()
eth_price_OSM['ETH Price (OSM)'] = eth_price_OSM['newMedian'].astype(float)/constant
# subset
eth_price_OSM = eth_price_OSM[['block_number','ETH Price (OSM)']]

eth_price_OSM.head()

,block_number,ETH Price (OSM)
0,12375195,3352.910000
1,12374951,3355.820670
2,12374655,3376.582733
3,12374385,3373.697700
4,12374112,3380.394789


In [4]:
# blocknumbers = eth_price_OSM.block_number.values.tolist()


# uniswap_url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
# pair = []
# for i in blocknumbers[-5:-1]:
#     query = '''
#     {
#       pairs(block: {number:%s}, where: {id: "0x8ae720a71622e824f576b4a8c03031066548a3b1"}){
#         id,
#         token0 {
#           id
#         },
#         token1 {
#           id
#         }
#         reserve0,
#         reserve1,
#         totalSupply,
#         reserveETH,
#         reserveUSD,
#         token0Price,
#         token1Price
#       }
#     }
    
#     ''' % i
#     r = requests.post(uniswap_url, json = {'query':query})
#     s = json.loads(r.content)['data']['pairs'][0]
#     pair.append(s)
        
# pairState = pd.DataFrame(pair)

# #pairState['block_number'] = blocknumbers


# pairState.head()



In [5]:
blocknumbers = eth_price_OSM.block_number.values.tolist()

state = []
for i in blocknumbers[:-5]:
    query = '''
    {
      systemState(block: {number:%s},id:"current") { 
        coinUniswapPair {
          reserve0
          reserve1
        }
        currentCoinMedianizerUpdate{
          value
        }
        currentRedemptionRate {
          eightHourlyRate
          annualizedRate
          hourlyRate
          createdAt
        }
        currentRedemptionPrice {
          value
        }
        erc20CoinTotalSupply
        globalDebt
        globalDebtCeiling
        safeCount,
        totalActiveSafeCount
        coinAddress
        wethAddress
        systemSurplus
        debtAvailableToSettle
        lastPeriodicUpdate
        createdAt
        createdAtBlock
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['systemState']
    state.append(s)
        
systemState = pd.DataFrame(state)

systemState['block_number'] = blocknumbers[:-5]


systemState.head()


,coinAddress,coinUniswapPair,createdAt,createdAtBlock,currentCoinMedianizerUpdate,currentRedemptionPrice,currentRedemptionRate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,lastPeriodicUpdate,safeCount,systemSurplus,totalActiveSafeCount,wethAddress,block_number
0,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '13922591.079704607464754936', 'r...",1529678381,5834717,{'value': '3.030441878636386318'},{'value': '3.00630484164175431089587452'},{'annualizedRate': '0.962694164312473697110361...,0,26348934.85601122035773477,26654970.29241748726163248578054693,115792089237316195423570985008687.9,1620225492,1964,228549.4633442294355009452623855005,590,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12375195
1,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '13930202.044450473028037132', 'r...",1529678381,5834717,{'value': '3.030441878636386318'},{'value': '3.00631588512580102437227149'},{'annualizedRate': '0.962694164312473697110361...,0,26348934.85601122035773477,26654970.29241748726163248578054693,115792089237316195423570985008687.9,1620225492,1964,228549.4633442294355009452623855005,590,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12374951
2,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '13939188.891627969262161539', 'r...",1529678381,5834717,{'value': '3.030441878636386318'},{'value': '3.006330067381913276425221243'},{'annualizedRate': '0.962694164312473697110361...,0,26318734.85601122035773477,26624780.6012696518453429543076774,115792089237316195423570985008687.9,1620216509,1964,228491.4823273391464605006380169398,590,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12374655
3,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '13940049.805792666093979086', 'r...",1529678381,5834717,{'value': '3.030441878636386318'},{'value': '3.006347795537092902390413548'},{'annualizedRate': '0.948181829489408201985626...,0,26319536.916318466781604157,26625579.1486517284524437123076774,115792089237316195423570985008687.9,1620216509,1964,228491.4823273391464605006380169398,592,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12374385
4,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '13940049.805792666093979086', 'r...",1529678381,5834717,{'value': '3.030441878636386318'},{'value': '3.00636609191444842653765813'},{'annualizedRate': '0.948181829489408201985626...,0,26319536.916318466781604157,26626058.40818813197633826355195016,115792089237316195423570985008687.9,1620216509,1964,228397.4121292081223849076637278554,592,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12374112


In [6]:
systemState.tail()

,coinAddress,coinUniswapPair,createdAt,createdAtBlock,currentCoinMedianizerUpdate,currentRedemptionPrice,currentRedemptionRate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,lastPeriodicUpdate,safeCount,systemSurplus,totalActiveSafeCount,wethAddress,block_number
1894,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",1529678381,5834717,{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.696064173575505967822069085347,115792089237316195423570985008687.9,1613409220,3,0.1278156009542366213422373102764649,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11862898
1895,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",1529678381,5834717,{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.696064173575505967822069085347,115792089237316195423570985008687.9,1613405255,3,0.1278156009542366213422373102764649,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11862600
1896,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",1529678381,5834717,{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.696064173575505967822069085347,115792089237316195423570985008687.9,1613405255,3,0.1278156009542366213422373102764649,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11862330
1897,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",1529678381,5834717,{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,1613394096,3,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11862059
1898,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",1529678381,5834717,{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,1613394096,3,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861791


In [7]:
systemState.coinUniswapPair.values[0]

{'reserve0': '13922591.079704607464754936',
 'reserve1': '12649.052425756721584902'}

In [8]:
systemState.currentCoinMedianizerUpdate.values[0]

{'value': '3.030441878636386318'}

In [9]:
systemState['RedemptionRateAnnualizedRate'] = systemState.currentRedemptionRate.apply(lambda x: x['annualizedRate'])    
systemState['RedemptionRateHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['hourlyRate'])
systemState['RedemptionRateEightHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['eightHourlyRate'])
systemState['RedemptionPrice'] = systemState.currentRedemptionPrice.apply(lambda x: x['value'])
systemState['RAIInUniswapV2(RAI/ETH)'] = systemState.coinUniswapPair.apply(lambda x: x['reserve0'])
del systemState['currentRedemptionRate']
del systemState['currentRedemptionPrice']
systemState['RedemptionRateAnnualizedRate'] = systemState['RedemptionRateAnnualizedRate'].astype(float)
systemState['RedemptionRateHourlyRate'] = systemState['RedemptionRateHourlyRate'].astype(float)
systemState['RedemptionRateEightHourlyRate'] = systemState['RedemptionRateEightHourlyRate'].astype(float)
systemState['RedemptionPrice'] = systemState['RedemptionPrice'].astype(float)
systemState['RAIInUniswapV2(RAI/ETH)'] = systemState['RAIInUniswapV2(RAI/ETH)'].astype(float)


# subset
systemState = systemState[['debtAvailableToSettle','erc20CoinTotalSupply','globalDebt',
                           'globalDebtCeiling','systemSurplus','totalActiveSafeCount',
                           'block_number', 'RedemptionRateAnnualizedRate',
                           'RedemptionRateHourlyRate', 'RedemptionRateEightHourlyRate',
                        'RedemptionPrice', 'RAIInUniswapV2(RAI/ETH)']]

In [10]:
#       hourlyStats(block: {number:%s}) { 


hourly = []
for i in blocknumbers[1:-5]:
    query = '''
    {
      hourlyStats(where: {blockNumber_gt: %s}) { 
        marketPriceUsd # price of COIN in USD (uni pool price * ETH median price)
        marketPriceEth # Price of COIN in ETH (uni pool price)
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['hourlyStats'][0]
    hourly.append(s)
        
hourlyStats = pd.DataFrame(hourly)

hourlyStats['block_number'] = blocknumbers[1:-5]

hourlyStats.head()


,marketPriceEth,marketPriceUsd,block_number
0,0.0009082420267933858122250685351943397,3.028273174804070426088451481937937,12374951
1,0.0009096462100114557443970573442790005,3.01554549915381725488420556681218,12374655
2,0.0009096462100114557443970573442790005,3.01554549915381725488420556681218,12374385
3,0.0009096462100114557443970573442790005,3.01554549915381725488420556681218,12374112
4,0.0009048101957348139921223497187843389,3.05255607623280316340690052533103,12373874


In [11]:
hourlyStats.describe()

,block_number
count,1.898000e+03
mean,1.211850e+07
std,1.482883e+05
min,1.186179e+07
25%,1.199025e+07
50%,1.211852e+07
75%,1.224696e+07
max,1.237495e+07


In [12]:
safehistories = []
for i in blocknumbers[:-5]:
    query = '''
    {
      safes(block: {number:%s}) {
            collateral
            debt
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['safes']
    t = pd.DataFrame(s)
    t['collateral'] = t['collateral'].astype(float)
    t['debt'] = t['debt'].astype(float)
    safehistories.append(pd.DataFrame(t.sum().to_dict(),index=[0]))

safe_history = pd.concat(safehistories)


In [13]:
safe_history.head()

,collateral,debt
0,1249.4226,293469.366076
0,1249.4226,293469.366076
0,1249.4226,293469.366076
0,1249.4226,293469.366076
0,1249.4226,293469.366076


In [14]:
safe_history['block_number'] = blocknumbers[:-5]
safe_history.reset_index(inplace=True)
del safe_history['index']

In [15]:
states = hourlyStats.merge(systemState,how='inner',on='block_number')
states = states.merge(safe_history,how='inner',on='block_number')

In [16]:
# prices = .merge(eth_price_next_OSM,how='inner',on='block_number')
# prices.head()

In [17]:
states = states.merge(eth_price_OSM,how='inner',on='block_number')

In [18]:
states.head()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RAIInUniswapV2(RAI/ETH),collateral,debt,ETH Price (OSM)
0,0.0009082420267933858122250685351943397,3.028273174804070426088451481937937,12374951,0,26348934.85601122035773477,26654970.29241748726163248578054693,115792089237316195423570985008687.9,228549.4633442294355009452623855005,590,0.962694,0.999996,0.999965,3.006316,1.393020e+07,1249.4226,293469.366076,3355.820670
1,0.0009096462100114557443970573442790005,3.01554549915381725488420556681218,12374655,0,26318734.85601122035773477,26624780.6012696518453429543076774,115792089237316195423570985008687.9,228491.4823273391464605006380169398,590,0.962694,0.999996,0.999965,3.006330,1.393919e+07,1249.4226,293469.366076,3376.582733
2,0.0009096462100114557443970573442790005,3.01554549915381725488420556681218,12374385,0,26319536.916318466781604157,26625579.1486517284524437123076774,115792089237316195423570985008687.9,228491.4823273391464605006380169398,592,0.948182,0.999994,0.999951,3.006348,1.394005e+07,1249.4226,293469.366076,3373.697700
3,0.0009096462100114557443970573442790005,3.01554549915381725488420556681218,12374112,0,26319536.916318466781604157,26626058.40818813197633826355195016,115792089237316195423570985008687.9,228397.4121292081223849076637278554,592,0.948182,0.999994,0.999951,3.006366,1.394005e+07,1249.4226,293469.366076,3380.394789
4,0.0009048101957348139921223497187843389,3.05255607623280316340690052533103,12373874,0,26459536.916318466781604157,26765446.21039985888196434955195016,115792089237316195423570985008687.9,228397.4121292081223849076637278554,592,0.948182,0.999994,0.999951,3.006384,1.395508e+07,1249.4226,293469.366076,3389.969206


In [19]:
states.tail()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RAIInUniswapV2(RAI/ETH),collateral,debt,ETH Price (OSM)
1893,0.001768934955701819449591545208406114,3.212255638445860905952903939205175,11862898,0,3048.440432971667032725,3048.696064173575505967822069085347,115792089237316195423570985008687.9,0.1278156009542366213422373102764649,3,1.0,1.0,1.0,3.14,801.933727,16.89236,3048.35849,1825.887144
1894,0.001768934955701819449591545208406114,3.223317461414449337259100162066151,11862600,0,3048.440432971667032725,3048.696064173575505967822069085347,115792089237316195423570985008687.9,0.1278156009542366213422373102764649,3,1.0,1.0,1.0,3.14,801.933727,16.89236,3048.35849,1793.770282
1895,0.001768934955701819449591545208406114,3.223317461414449337259100162066151,11862330,0,3048.440432971667032725,3048.696064173575505967822069085347,115792089237316195423570985008687.9,0.1278156009542366213422373102764649,3,1.0,1.0,1.0,3.14,801.933727,16.89236,3048.35849,1803.891149
1896,0.001768934955701819449591545208406114,3.215446177235759811208395560891868,11862059,0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,1.0,1.0,1.0,3.14,801.933727,16.89236,3048.35849,1803.891149
1897,0.001768934955701819449591545208406114,3.215446177235759811208395560891868,11861791,0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,1.0,1.0,1.0,3.14,801.933727,16.89236,3048.35849,1805.792735


In [20]:
states = states.astype(float)

In [21]:
# export
states.to_csv('states.csv')